- This notebook is to check if different normalization(log) for weight will affect results

In [1]:
import sys
sys.path.append("../src")
sys.path.append("../")

In [6]:
from VICGAE_latent import *
import pandas as pd
import numpy as np

In [3]:
from astrochem_embedding import VICGAE

model = VICGAE.from_pretrained()

/Users/haoliu/anaconda3/envs/VICGAE_/lib/python3.9/pickle.py:1579: DeprecationWarning: `torch.distributed._sharded_tensor` will be deprecated, use `torch.distributed._shard.sharded_tensor` instead
  __import__(module, level=0)


In [7]:
def normalize_concentrations_log(df, conc_cols):
    for conc in conc_cols:
        df[conc] = np.log1p(df[conc])  
    return df

In [8]:
df_mixture = pd.read_csv('df_mixture_20250502.csv')
# Normalize concentrations
conc_cols = [f'conc{i}' for i in range(1, 3)]
df_mixture_norm = normalize_concentrations_log(df_mixture, conc_cols)
df_mixture_norm.head()

,smi1,conc1,smi2,conc2,miscibility
0,C[C@H](CCC(=O)NCCC[N+](C)(C)CCCS(=O)(=O)[O-])[...,5.097596,CC(C)(C)CC(C)(C)C1=CC=C(C=C1)OCCO,5.471754,1
1,C[C@H](CCC(=O)NCCC[N+](C)(C)CCCS(=O)(=O)[O-])[...,5.097596,[C@H]1([C@@H]([C@@H](OC([C@H]1O)C(=O)O)O[C@H]2...,3.819756,1
2,C[C@H](CCC(=O)NCCC[N+](C)(C)CCCS(=O)(=O)[O-])[...,5.097596,C([C@@H]1[C@H]([C@@H]([C@H]([C@H](O1)O[C@@H]2[...,4.116358,1
3,C[C@H](CCC(=O)NCCC[N+](C)(C)CCCS(=O)(=O)[O-])[...,5.097596,C1[C@@H]2[C@@H]([C@H](O1)[C@H]([C@H](O2)O)OS(=...,2.196230,1
4,C[C@H](CCC(=O)NCCC[N+](C)(C)CCCS(=O)(=O)[O-])[...,5.097596,C1[C@@H]2[C@@H]([C@H](O1)[C@H]([C@H](O2)O)O)O[...,1.993277,1


In [9]:
x_latent, y = build_mixture_latent_features_VICGAE(
    df=df_mixture_norm,
    smi_cols=['smi1', 'smi2'],
    conc_cols=['conc1', 'conc2'],
    target_col='miscibility',
    latent_fn=latent_fn_VICGAE,
    latent_fn_args={"model": model}
)


Skipped 1 SMILES due to errors:
  - [3H]C(CC[C@@H](C(=O)NCC(=O)N[C@@H](CC1C=NC=N1)C(=O)N[C@@H](CCCN=C(N)N)C(=O)NCC(=O)N[C@@H](CC2=CC=CC=C2)C(=O)N[C@@H](CO)C(=O)NCC(=O)OC(=O)[C@H](CC(C)C)NC(=O)CN)NC(=O)[C@H](CCCC)NC(=O)CNC(=O)[C@@H]3C[C@H](CN3)O)C([3H])N: '[3H]'


In [10]:
x_latent.shape

(1326, 32)

In [11]:
df_total = pd.concat([pd.DataFrame(x_latent),pd.DataFrame(y, columns=['miscibility'])], axis=1)
df_total

,0,1,2,3,4,5,6,7,8,9,...,23,24,25,26,27,28,29,30,31,miscibility
0,-0.356866,-1.561678,2.415661,0.964029,-2.870945,1.831822,-0.424897,0.732239,1.504168,-2.594391,...,1.939532,-2.620416,-2.091833,-1.612540,2.670709,1.788816,-1.477206,3.125855,-0.714750,1
1,-0.090276,-0.749380,2.493162,1.047147,-2.743200,2.359014,-0.654413,0.896836,2.012722,-2.307658,...,1.683482,-2.828351,-1.254388,-1.518992,2.135478,1.398648,-1.719716,3.160266,-1.430736,1
2,0.297342,-0.422899,2.695721,0.473949,-4.067978,3.622295,-1.085541,1.580660,1.843884,-2.829342,...,1.512648,-4.085516,-1.167193,-1.247433,1.820917,1.911202,-1.778385,4.798835,-3.190676,1
3,-0.458573,-0.560705,2.880292,1.066262,-3.119522,2.701172,-1.057606,1.265694,1.951892,-2.277787,...,2.043437,-3.065849,-1.773978,-1.675073,2.367247,1.065733,-1.663210,3.455431,-1.682603,1
4,-0.368420,-0.518194,2.641258,0.828455,-2.914365,2.502261,-0.992847,1.268153,1.674948,-2.124714,...,1.863178,-2.798311,-1.732437,-1.484397,2.092368,1.019408,-1.534466,3.198484,-1.614147,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1321,0.250091,-1.242906,1.313321,1.360490,-1.487503,1.065170,0.763994,0.006870,1.613358,-1.339026,...,0.788468,-1.442180,-0.804905,-1.081854,1.493656,1.057494,-0.393116,1.919353,0.190541,1
1322,-0.263044,-0.973998,0.634356,0.655439,-0.992068,0.540900,0.514133,-0.197979,0.737419,-0.695712,...,0.296455,-0.903724,-0.675177,-0.268493,0.760633,0.723723,0.321379,1.563382,0.252702,1
1323,0.177190,-0.464731,2.488227,1.086583,-2.394904,1.958475,-0.523807,1.091688,1.488105,-1.602476,...,1.757111,-2.127674,-1.500801,-1.465427,1.564054,0.963596,-0.992481,2.269214,-0.934379,0
1324,-0.335944,-0.195823,1.809262,0.381531,-1.899470,1.434205,-0.773668,0.886839,0.612165,-0.959162,...,1.265097,-1.589218,-1.371073,-0.652066,0.831031,0.629826,-0.277987,1.913243,-0.872218,1


In [12]:
df_total.to_csv('df_total_binary_VICGAE_log_20250930.csv', index=False)